In [6]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from glob import glob
import cv2
import random
import os

In [8]:
from tensorflow.keras.utils import Sequence
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
class CustomDataGenerator(Sequence):
    def __init__(self, file_paths, labels, batch_size, target_size):
        self.file_paths = file_paths
        self.labels = labels
        self.batch_size = batch_size
        self.target_size = target_size

    def __len__(self):
        return int(np.ceil(len(self.file_paths) / self.batch_size))

    def __getitem__(self, index):
        batch_files = self.file_paths[index * self.batch_size: (index + 1) * self.batch_size]
        batch_labels = self.labels[index * self.batch_size: (index + 1) * self.batch_size]

        batch_images = [np.array(Image.open(file).resize(self.target_size)) / 255.0 for file in batch_files]

        return np.array(batch_images), np.array(batch_labels)


wrinkled_dataset_path = 'datasets/train/Wrinkled'
non_wrinkled_dataset_path = 'datasets/train/NoWrinkles'


wrinkled_files = glob(os.path.join(wrinkled_dataset_path, '*.jpg'))


wrinkled_labels = [1] * len(wrinkled_files)


non_wrinkled_files = glob(os.path.join(non_wrinkled_dataset_path, '*.jpg'))


non_wrinkled_labels = [0] * len(non_wrinkled_files)


all_files = wrinkled_files + non_wrinkled_files
all_labels = wrinkled_labels + non_wrinkled_labels


train_files, val_files, train_labels, val_labels = train_test_split(
    all_files, all_labels, test_size=0.2, random_state=42
)


train_datagen = CustomDataGenerator(train_files, train_labels, batch_size=batch_size, target_size=(128, 128))
validation_datagen = CustomDataGenerator(val_files, val_labels, batch_size=batch_size, target_size=(128, 128))


model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.fit(train_datagen, epochs=10, validation_data=validation_datagen)

Epoch 1/10
7/7 [==============================] - 12s 1s/step - loss: 0.5996 - accuracy: 0.7321 - val_loss: 0.6012 - val_accuracy: 0.7358
Epoch 2/10
7/7 [==============================] - 8s 1s/step - loss: 0.5818 - accuracy: 0.7608 - val_loss: 0.6719 - val_accuracy: 0.7358
Epoch 3/10
7/7 [==============================] - 8s 1s/step - loss: 0.5687 - accuracy: 0.7608 - val_loss: 0.5338 - val_accuracy: 0.7358
Epoch 4/10
7/7 [==============================] - 7s 980ms/step - loss: 0.5274 - accuracy: 0.7608 - val_loss: 0.5096 - val_accuracy: 0.7358
Epoch 5/10
7/7 [==============================] - 7s 998ms/step - loss: 0.5170 - accuracy: 0.7608 - val_loss: 0.5258 - val_accuracy: 0.7358
Epoch 6/10
7/7 [==============================] - 7s 1s/step - loss: 0.5436 - accuracy: 0.7608 - val_loss: 0.4949 - val_accuracy: 0.7358
Epoch 7/10
7/7 [==============================] - 7s 957ms/step - loss: 0.5435 - accuracy: 0.7608 - val_loss: 0.4707 - val_accuracy: 0.7358
Epoch 8/10
7/7 [===============

In [9]:
eval_result = model.evaluate(validation_datagen)
print(f"Validation Accuracy: {eval_result[1] * 100:.2f}%")

2/2 [==============================] - 1s 167ms/step - loss: 0.4835 - accuracy: 0.7358
Validation Accuracy: 73.58%


In [10]:
model.save('wrinkle_model.h5')

C:\Users\DELL\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 63, 63, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 30, 30, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 28, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 14, 14, 128)       0

In [12]:
model_json = model.to_json()
with open("wrinkle.json","w") as json_file:
    json_file.write(model_json)